In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install -U sentence-transformers


In [3]:
pip install elasticsearch

In [4]:
import elasticsearch
print(elasticsearch.__version__)

(8, 12, 0)


In [5]:
import time
import sys
import json
import random
import os
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import numpy as np
from itertools import islice


In [6]:
class Search:
    def __init__(self):
        PORT="https://0840-5-160-41-141.ngrok-free.app"
        AUTH=("elastic", "be_90Z5Su9WhYylUwOir")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch(PORT,basic_auth=AUTH,verify_certs=False)
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)

    def get_embedding(self, text):
        return self.model.encode(text)

    def insert_document(self, index, document):
        return self.es.index(index=index, document={
            **document,
            'embedding': self.get_embedding(document['Name']),
        })

    def insert_documents(self, index, documents):
        for document in documents:
            self.insert_document(index,document)

    def insert_LLM_knowledge(self, index, documents):
      for document in documents:
            return self.es.index(index=index, document={
                **document,
                'embedding': self.get_embedding(document['Text']),
            })

    def search(self, index, **query_args):
        return self.es.search(index=index, **query_args)

    def delete_index(self, index):
      return self.es.indices.delete(index=index, ignore=[400, 404])

    def create_embedding_index(self,index):
        self.es.indices.delete(index=index, ignore_unavailable=True)
        self.es.indices.create(index=index, mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

In [7]:
es = Search()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://0840-5-160-41-141.ngrok-free.app:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: http

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ClOJj4yGRH6j6iWJSJd9yw',
 'name': 'cc5825acbd1d',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-12-08T11:33:53.634979452Z',
             'build_flavor': 'default',
             'build_hash': '64cf052f3b56b1fd4449f5454cb88aca7e739d9a',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.8.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.11.3'}}


In [8]:
! cp /content/drive/MyDrive/ColabNotebooks/json.json /content/json.json

In [9]:
with open('json.json', 'rt') as f:
    documents = json.loads(f.read())

In [10]:
es.create_embedding_index('all_documents')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
es.insert_documents('all_documents',documents)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/p

In [12]:
query = "KODAK"
es.search('all_documents',
    query={
        'match': {
            'Name': {
                'query': query
            }
        }
    }
)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'took': 67, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 4.7861624, 'hits': [{'_index': 'all_documents', '_id': 'Jsq6dY0BElHoeBVYRlNf', '_score': 4.7861624, '_ignored': ['Reviews.keyword'], '_source': {'Name': 'Kodak AZ401RD Point & Shoot Digital Camera with 3" LCD, Red', 'Price': '169.99', 'Details': ['You’ll get well defined, detailed images thanks to the AZ401’s 16 million pixels. That’s ample resolution to crop, zoom, or enlarge your photos however the photographer in you desires.', 'With a 24mm wide angle lens, there’s no sacrificing what you want in the frame. So go ahead and let your camera see what you see - the sky’s the limit!', 'The Optical Image Stabilization will iron out any small movements as you’re focused on getting your shot, so you never have to be afraid to go for what you want.', 'Whether you’re in the moment, or modifying your work at a lat

In [13]:
query = "KODAK"
es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10, 'relation': 'eq'}, 'max_score': 0.7760743, 'hits': [{'_index': 'all_documents', '_id': 'Qcq6dY0BElHoeBVYblP_', '_score': 0.7760743, '_ignored': ['Reviews.keyword'], '_source': {'Name': 'KODAK PIXPRO FZ55-BK 16MP Digital Camera 5X Optical Zoom 28mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (Black)', 'Price': '119.99', 'Details': ['16 Megapixel CMOS Sensor', '5X Optical Zoom - 28mm Wide Angle Lens', '1080P Full HD Video - Vlogging Camera', 'SD Card Compatibility: At least Class 10, no more than 512GB (SD, SDHC, SDXC)', '2.7" LCD Screen - Rechargeable Li-Ion Battery'], 'Ratings': '4.2', 'Rates': '842', 'Reviews': ["Bought this camera for a simple camera to use while traveling when I didn't want to take my huge Panasonic camera but still wanted a camera that wasn't on my phone, for safety purposes and also the anxiety of possi

In [14]:
def Recommender(index,selected_doc):
    results = es.search(index,
        query={
            'multi_match': {
                'query': selected_doc['Name'],
                'fields': ['Name', 'Details'],
            }
        }
    )
    return results['hits']['hits']



In [15]:
Recommender('all_documents',documents[2])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'_index': 'all_documents',
  '_id': 'Csq6dY0BElHoeBVYFFN6',
  '_score': 45.71621,
  '_ignored': ['Details.keyword', 'Reviews.keyword', 'Link.keyword'],
  '_source': {'Name': 'Digital Camera, Kids Camera FHD 1080P 44MP Small Digital Camera with 32GB SD Card Vlogging Camera 16X Digital Zoom, Point and Shoot Digital Camera Gift for Kids Boys Girls Teens Students - Black',
   'Price': '36.79',
   'Details': ['[FHD 1080P & 44MP Camera]: This digital camera cheap with 44 MP picture pixels that can shoot 1080p videos, will help you get exciting photos or videos, and our kids digital camera comes with a 32GB SD card and additional features like face detection, smile capture, self timer (3S/5S/10S), 16X digital zoom, fill light, Anti- shake function etc, if you want a small digital cameras to vlog and take cute pics, it is great for memories.',
    "[Portable Travel Digital Camera]: Our compact digital camera is smaller than a smart phone, enough small to fit your pocket or backpack, suitable

In [16]:
selected_docs = random.sample(documents,k = 20)

In [17]:
es.insert_documents('selected_documents',selected_docs)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/p

In [18]:
adjacency_dictionary = {}
for document in selected_docs:
    adjacency_dictionary[document["Name"]] = Recommender('selected_documents',document)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/p

In [19]:
vertices = sorted(adjacency_dictionary.keys())
matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(matrix)


[[49.00164, 0, 10.5136595, 14.099749, 6.7409186, 9.664058, 8.209989, 0, 0, 7.9122653, 11.265357, 5.5490108, 9.366303, 0, 0, 0, 0, 0, 0], [4.7504425, 52.04716, 4.30123, 0, 0, 3.454742, 0, 10.289786, 7.6658735, 0, 4.1018634, 20.850628, 0, 0, 0, 21.70219, 0, 16.108452, 0], [8.139524, 0, 41.892357, 20.55714, 6.6211305, 13.550332, 15.168052, 0, 0, 9.5894, 13.356192, 0, 12.159942, 0, 0, 6.550754, 0, 0, 0], [11.379446, 0, 21.008163, 39.40838, 14.02589, 17.615446, 11.967241, 0, 0, 15.939419, 19.465582, 0, 13.388566, 0, 14.742925, 0, 0, 0, 0], [11.132822, 10.461378, 13.88147, 19.520746, 42.282284, 25.817913, 12.054525, 0, 0, 26.968626, 21.438416, 0, 0, 0, 26.268827, 0, 0, 0, 0], [10.103375, 6.109764, 10.045508, 12.903487, 17.1147, 38.71662, 0, 0, 5.1101885, 20.423546, 14.938207, 0, 0, 0, 17.1147, 0, 0, 0, 0], [8.507082, 0, 12.694629, 12.572963, 6.3136306, 13.353431, 48.30488, 0, 11.162197, 8.876446, 13.495488, 0, 25.296751, 0, 0, 0, 0, 0, 0], [0, 10.434702, 0, 4.435609, 0, 5.801995, 0, 47.50694

In [20]:
query = "Vlogging Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


4K Digital Camera for Photography 48MP Vlogging Camera for YouTube Vloggers Gifts for Beginners Teens with Rechargeable Battery
VJIANGER 2024 Upgrade 4K Digital Camera for Photography Vlogging Camera for YouTube 64MP Photography Camera with WiFi, 52mm Fixed Lens, 4.0" Touch Screen, 32GB SD Card & Camera Bag(W05-BlackS2)
VJIANGER 2024 Upgrade 4K Digital Camera for Photography Vlogging Camera for YouTube 64MP Photography Camera with WiFi, 52mm Fixed Lens, 4.0" Touch Screen, 32GB SD Card & Camera Bag(W05-BlackS2)
VJIANGER 4K Digital Camera 48MP WiFi Vlogging Camera for YouTube with 180° Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black4)
Digital Camera for Photography VJIANGER 4K 48MP Vlogging Camera for YouTube with WiFi, 180 Degree Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black6)
NIKICAM Digital Camera for Photography 4K 56MP Vlogging Camera for YouTube with 16X Digital Zoom, 2 Rechargeable 

In [21]:
query = "Photography"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
VETEK Digital Cameras for Photography, 4K 48MP Vlogging Camera 16X Digital Zoom Manual Focus Students Compact Camera with 52mm Wide-Angle Lens & Macro Lens, 32G Micro Card and 2 Batteries (Pink)
KODAK PIXPRO AZ405-BK 20MP Digital Camera 40X Optical Zoom 24mm Wide Angle Lens Optical Image Stabilization 1080P Full HD Video 3" LCD Vlogging Camera (Black)
Digital Camera, Auto Focus FHD 4K Vlogging Camera with Dual Camera 48MP 16X Digital Zoom Kids Compact Camera with 32GB Memory Card Portable Point and Shoot Cameras for Teens Beginner Adult,Black
Digital Camera, 48MP HD Autofocus Anti-Shake 18X Digital Zoom V

In [22]:
query = "video camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


G-Anica Digital Camera 4K Camcorder 48MP 3.0" IPS Flip Screen Video Camera 16X Digital Zoom Vlogging Camera for YouTube, Compact Camera with 32GB SD Card and Wide Angle Lens Macro Lens…
G-Anica 4K Digital Cameras for Photography，48MP/60FPS Video Camera for Vlogging, WiFi & App Control Vlogging Camera for YouTube, Small Camera with 32GB TF Card.Wide-Angle & Macro Lens
KODAK PIXPRO FZ45-WH 16MP Digital Camera 4X Optical Zoom 27mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (White)
KODAK PIXPRO FZ55-BK 16MP Digital Camera 5X Optical Zoom 28mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (Black)
Digital Camera for Photography VJIANGER 4K 48MP Vlogging Camera for YouTube with WiFi, 180 Degree Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black6)
4K Digital Camera for Photography 48MP Vlogging Camera for YouTube Vloggers Gifts for Beginners Teens with Rechargeable Battery
VJIANGER 4K Digital Camera 48MP WiFi Vlogging Camera 

In [23]:
query = "Kids Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera, Kids Camera FHD 1080P 44MP Small Digital Camera with 32GB SD Card Vlogging Camera 16X Digital Zoom, Point and Shoot Digital Camera Gift for Kids Boys Girls Teens Students - Black
GREENKINDER Upgrade Kids Camera Christmas Birthday Gifts for Boys Girls Age 3-12, Children Digital Video Cameras with Flash, 2 Inch Screen Selfie Camcorder HD 1080P Video 32GB Black
MEETRYE Instant Print Camera for Kids - Christmas and Birthday Gift Ideas - Digital Toy Camera for Girls and Boys Ages 4 to 14
Instant Print Camera - PROGRACE Kids Camera Toy 1080P HD 0 Ink Print Photo Christmas Birthday Gifts for Girl

In [24]:
query = "Camera 4K"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


4K Digital Camera for Photography Auto-Focus 4K Camera with 180° 3.0 inch Flip Screen 16X Anti-Shake Vlogging Camera for YouTube Video Compact Cameras with SD Card, 2 Batteries and Battery Charger
4K Digital Camera for Photography Auto-Focus 4K Camera with 180° 3.0 inch Flip Screen 16X Anti-Shake Vlogging Camera for YouTube Video Compact Cameras with SD Card, 2 Batteries and Battery Charger
Newest 4K Digital Camera for Photography and Video, 48MP Vlogging Camera with SD Card Autofocus Anti-Shake, 3'' 180° Flip Screen 16X Zoom Digital Camera with Flash, Compact Digital Camera for Travel
4K Digital Camera for Photography Autofocus, Upgraded 48MP Vlogging Camera for YouTube with SD Card, 3" 180 Flip Screen Compact Travel Camera with 16X Digital Zoom, Flash, Anti-Shake, 2 Batteries
YEEIN 4K Cameras for Photography and Video, 3'' Screen Compact Digital Camera with Leather Case, Autofocus 48MP Vlogging Camera with 32G Card, Dual Lens 4k Camera with Built-in 7 Color Filters
4K Digital Cameras

In [25]:
testing_documents = documents[:5]

In [26]:
es.insert_documents('testing_documents',testing_documents)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/p

In [27]:

adjacency_dictionary = {}
for document in testing_documents:
    adjacency_dictionary[document["Name"]] = Recommender('testing_documents',document)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0840-5-160-41-141.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/p

In [28]:
vertices = sorted(adjacency_dictionary.keys())
prediction_matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(prediction_matrix)

[[25.103662, 6.4932923, 5.81129, 4.9018664, 11.566331], [13.581725, 24.47444, 1.9819951, 4.6392694, 7.939229], [2.54001, 1.7531276, 18.232708, 10.275377, 1.8881803], [0, 1.8904148, 0, 18.60863, 1.7185338], [8.770007, 6.4932923, 0.9791776, 4.84325, 14.817237]]


In [29]:
ground_truth = [[1,4,3,5,2],
                [2,1,5,4,3],
                [3,4,1,2,5],
                [3,5,2,1,4],
                [2,3,5,4,1]]

In [30]:
def rank_predictions(prediction_matrix):
    ranked_predictions = []
    for i, row in enumerate(prediction_matrix):
        ranked_predictions.append(row)
        ranked_predictions[i] = sorted(ranked_predictions[i], reverse = True)
        for j in range (len(prediction_matrix)):
            indx = ranked_predictions[i].index(row[j])
            row [j] = indx + 1
    return  prediction_matrix

def apk(actual, predicted, k):
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p == actual[i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    predicted = rank_predictions(predicted)
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [31]:
mapk(ground_truth,  prediction_matrix, 5)

0.5906666666666667

In [32]:
def DCG_at_k(rel_vector, k):
    rel_vector = np.asarray(rel_vector)[:k]
    discounts = np.log2(np.arange(len(rel_vector)) + 2)
    print("Discounts:", discounts)
    print("Rel Vector:", rel_vector)
    return np.sum((np.power(2, rel_vector) - 1) / discounts)

def NDCG_at_k(ground_truth, predictions, k):
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    rel_scores_ground_truth = ground_truth.flatten()
    rel_scores_predictions = predictions.flatten()

    DCG_ground_truth = DCG_at_k(rel_scores_ground_truth, k)
    DCG_predictions = DCG_at_k(rel_scores_predictions, k)

    ideal_ground_truth = np.sort(rel_scores_ground_truth)[::-1]
    IDCG_ground_truth = DCG_at_k(ideal_ground_truth, k)
    print("DCG Ground Truth:", DCG_ground_truth)
    print("DCG Predictions:", DCG_predictions)
    print("IDCG Ground Truth:", IDCG_ground_truth)

    if IDCG_ground_truth == 0:
        return 0.
    return DCG_predictions / IDCG_ground_truth

In [33]:
k = 5
ndcg = NDCG_at_k(ground_truth, rank_predictions(prediction_matrix), k)
print(ndcg)

Discounts: [1.         1.5849625  2.         2.32192809 2.5849625 ]
Rel Vector: [1 4 3 5 2]
Discounts: [1.         1.5849625  2.         2.32192809 2.5849625 ]
Rel Vector: [5 3 2 1 4]
Discounts: [1.         1.5849625  2.         2.32192809 2.5849625 ]
Rel Vector: [5 5 5 5 5]
DCG Ground Truth: 28.47547802555067
DCG Predictions: 43.149976941591724
IDCG Ground Truth: 91.40223268526115
0.47208887216329204


In [34]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved i

In [35]:
!git clone https://github.com/google/sentencepiece.git
!cd sentencepiece
!mkdir build
!cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

fatal: destination path 'sentencepiece' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install'.  Stop.
/sbin/ldconfig.real: Path `/usr/local/cuda-12/targets/x86_64-linux/lib' given more than once
(from /etc/ld.so.conf.d/988_cuda-12.conf:1 and /etc/ld.so.conf.d/000_cuda.conf:1)
/sbin/ldconfig.real: Path `/usr/local/cuda-12.2/targets/x86_64-linux/lib' given more than once
(from /etc/ld.so.conf.d/gds-12-2.conf:1 and /etc/ld.so.conf.d/000_cuda.conf:1)
/sbin/ldconfig.real: Path `/usr/local/lib' given more than once
(from /etc/ld.so.conf.d/libc.conf:2 and /etc/ld.so.conf.d/colab.conf:1)
/sbin/ldconfig.real: Can't stat /usr/local/nvidi

In [36]:
import torch
# print(torch.cuda.current_device())
print(torch.cuda.is_available())
print("Torch version:",torch.__version__)

False
Torch version: 2.1.0+cu121


In [37]:
!pip install optimum
!pip install auto-gptq
!pip install transformers

In [38]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install accelerate

In [39]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.0.3-py3-none-any.whl (819 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.28.1-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.3 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached distro-1.9.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-sacvhkyz/overlay/local/bin
    changing mode of /tmp/pip-build-env-sacvhkyz/overlay/local/bin/ninja to 755
    changing mod

In [40]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=2048, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,
    n_ctx=2048

    )


In [ ]:
lcpp_llm.params.n_gpu_layers

In [ ]:
prompt = "how much is a cannon camera?"
system_prompt = "Cannon Camera is expensive. prices is in range of 100 to 200 dollars"


prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. You know that {system_prompt}.

USER:  {prompt}

ASSISTANT:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [ ]:
print(response["choices"][0]["text"])

In [ ]:
query = "canon"
res = es.search('all_documents',
knn={
  'field': 'embedding',
  'query_vector': es.get_embedding(query),
  'num_candidates': 50,
  'k': 10,
},

)
res = res['hits']['hits']
print(res)

In [ ]:
def askLLM_context(prompt):
    context = ""
    query = f"{prompt}"
    res = es.search('all_documents',
            knn={
                'field': 'embedding',
                'query_vector': es.get_embedding(query),
                'num_candidates': 50,
                'k': 3,
            },

        )['hits']['hits']
    print(len(res))
    for i in res:
        context =context +"Name:" +i['_source']['Name']+"+"
        context =context +"Price:"+i['_source']['Price']+ "+"
        context =context +"Details:"+"".join(i['_source']['Details'][0])
    print(len(context))

    res = [context[idx:idx + 1000] for idx in range(len(context) - 1000 + 1)]
    print(len(res))
    LLM_documents = []
    for i in res:
      LLM_documents.append({'Name':i})
    print(len(LLM_documents))
    res = es.insert_documents('llm',LLM_documents)
    res = es.search('llm',
            knn={
                'field': 'embedding',
                'query_vector': es.get_embedding(query),
                'num_candidates': 50,
                'k': 2,
            },

        )['hits']['hits']
    LLM_context = ""

    for i in res:
        LLM_context =LLM_context + i['_source']['Name']+ "  "
    prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. You know that {LLM_context}.

    USER:  {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    print(response["choices"][0]["text"])
    es.delete_documents('llm')


In [ ]:
ask_LLM("i want to buy a canon camera with price under 100 dollars")

In [ ]:
ask_LLM("i want to buy a digital camera")

In [ ]:
ask_LLM("i want to buy a 4k camera")

In [ ]:
ask_LLM("i want to buy a waterproof camera")

In [ ]:
ask_LLM("i want to buy a vlogging camera")